In [ ]:
from hardware_testing.gravimetric.helpers import get_api_context
from opentrons_shared_data.liquid_classes.liquid_class_definition import PositionReference
from opentrons.protocol_api import InstrumentContext, Well

In [ ]:
# DEFINE SLOTS

SLOTS = {
    "trash": "A3",
    "t50": "B1",
    "t200": "B2",
    "t1000": "B3",
    "water": "C1",
    "glycerol_50": "C2",
    "ethanol_80": "C3",
}

In [ ]:
# PROTOCOL CONTEXT (skip LPC offsets)

ctx = get_api_context(include_labware_offsets=False)

In [ ]:
# SETUP PIPETTES

PIPETTES = {
    50: ctx.load_instrument(
        instrument_name="flex_1channel_50",
        mount="left",
    ),
    1000: ctx.load_instrument(
        instrument_name="flex_1channel_1000",
        mount="right",
    ),
}

In [ ]:
# SETUP TIPS

TIP_RACKS = {
    tip: ctx.load_labware(
        load_name=f"opentrons_flex_96_tiprack_{tip}ul",
        location=SLOTS[f"t{tip}"]
    )
    for tip in [50, 200, 1000]
}
ctx.load_trash_bin(SLOTS["trash"])


def _get_clean_tip(pipette: int, tip: int) -> None:
    for _pip in PIPETTES.values():
        _pip._retract()
    for _pip in PIPETTES.values():
        if _pip.has_tip:
            _pip.drop_tip()
    PIPETTES[pipette].tip_racks = [TIP_RACKS[tip]]
    if not PIPETTES[pipette].has_tip:
        PIPETTES[pipette].pick_up_tip()


In [ ]:
# SETUP LIQUIDS + VIALS

LIQUID_NAMES = ["water", "glycerol_50", "ethanol_80"]
VIALS = {n: None for n in LIQUID_NAMES}


def _get_vial(liquid_name: str) -> Well:
    if not VIALS[liquid_name]:
        _vial_labware = ctx.load_labware(
            load_name="radwag_pipette_calibration_vial",
            location=SLOTS[liquid_name],
            version=2
        )
        _vial_labware.load_empty(_vial_labware.wells())
        VIALS[liquid_name] = _vial_labware["A1"]
    return VIALS[liquid_name]

In [ ]:
# SETUP TRIALS

VOLUMES = {
    50: {
        50: [1, 10, 50],
    },
    1000: {
        50: [5, 10, 50],
        200: [5, 50, 200],
        1000: [10, 100, 1000],
    }
}


def _transfer_liquid_qc_volumes(liquid_name: str, pipette: int, tip: int) -> None:
    # test all 3x volumes w/ same tip
    # tip SHOULD be empty for all combos
    # so new tips aren't required...
    _get_clean_tip(pipette, tip)
    _vial = _get_vial(liquid_name)
    PIPETTES[pipette].require_liquid_presence(_vial)
    # NOTE: Reloading definition each time allows us to:
    #         1) edit values in shared-data
    #         2) push-to to Flex running this notebook
    #         3) run another transfer (without restarting Kernel)
    liq_cls = ctx.define_liquid_class(liquid_name)
    for v in VOLUMES[pipette][tip]:
        PIPETTES[pipette].configure_for_volume(v)  # TODO: remove once added to API
        PIPETTES[pipette].prepare_to_aspirate()  # TODO: remove once added to API
        PIPETTES[pipette].transfer_liquid(
            liquid_class=liq_cls,
            volume=v,
            source=_vial,
            dest=_vial,
            new_tip="never"
        )


In [ ]:
raise RuntimeError("don't get careless and hit Run All")

In [ ]:
# HOME (optional)

ctx.home()

In [ ]:
# P50 + T50

In [ ]:
_transfer_liquid_qc_volumes("water", pipette=50, tip=50)

In [ ]:
_transfer_liquid_qc_volumes("glycerol_50", pipette=50, tip=50)

In [ ]:
_transfer_liquid_qc_volumes("ethanol_80", pipette=50, tip=50)

In [ ]:
# P1000 + T50

In [ ]:
_transfer_liquid_qc_volumes("water", pipette=1000, tip=50)

In [ ]:
_transfer_liquid_qc_volumes("glycerol_50", pipette=1000, tip=50)

In [ ]:
_transfer_liquid_qc_volumes("ethanol_80", pipette=1000, tip=50)

In [ ]:
# P1000 + T200

In [ ]:
_transfer_liquid_qc_volumes("water", pipette=1000, tip=200)

In [ ]:
_transfer_liquid_qc_volumes("glycerol_50", pipette=1000, tip=200)

In [ ]:
_transfer_liquid_qc_volumes("ethanol_80", pipette=1000, tip=200)

In [ ]:
# P1000 + T1000

In [ ]:
_transfer_liquid_qc_volumes("water", pipette=1000, tip=1000)

In [ ]:
_transfer_liquid_qc_volumes("glycerol_50", pipette=1000, tip=1000)

In [ ]:
_transfer_liquid_qc_volumes("ethanol_80", pipette=1000, tip=1000)